In [33]:
pip install mistralai


  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.20.1
    Uninstalling pydantic_core-2.20.1:
      Successfully uninstalled pydantic_core-2.20.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install PyMuPDF


   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 8.5 MB/s eta 0:00:02
   ----- ---------------------------------- 2.4/16.5 MB 8.9 MB/s eta 0:00:02
   ---------- ----------------------------- 4.5/16.5 MB 8.9 MB/s eta 0:00:02
   ------------- -------------------------- 5.8/16.5 MB 8.4 MB/s eta 0:00:02
   --------------- ------------------------ 6.3/16.5 MB 6.9 MB/s eta 0:00:02
   ------------------- -------------------- 8.1/16.5 MB 7.2 MB/s eta 0:00:02
   ----------------------- ---------------- 9.7/16.5 MB 7.4 MB/s eta 0:00:01
   --------------------------- ------------ 11.5/16.5 MB 7.7 MB/s eta 0:00:01
   ------------------------------- -------- 13.1/16.5 MB 7.6 MB/s eta 0:00:01
   ------------------------------------ --- 15.2/16.5 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 16.5/16.5 MB 7.8 MB/s eta 0:00:00
Note: you

In [2]:
from mistralai import Mistral

api_key = "uRZPeeEVZqxMDRYGqDbJIRabxsDFn8h0"
client = Mistral(api_key=api_key)

In [680]:
from pathlib import Path

pdf_file = Path("Downloads/data extraction/What Determines the Psychological Well-Being during Commute in Xi'an The Role of Built Environment, Travel Attitude, and Travel Characteristics.pdf")
assert pdf_file.is_file()

In [682]:
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import json

uploaded_file = client.files.upload(
    file={
        "file_name": pdf_file.stem,
        "content": pdf_file.read_bytes(),
    },
    purpose="ocr",
)

signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

pdf_response = client.ocr.process(document=DocumentURLChunk(document_url=signed_url.url), model="mistral-ocr-latest", include_image_base64=True)

response_dict = json.loads(pdf_response.json())
with open("output.json", "w", encoding="utf-8") as f:
    print(json.dump(response_dict, f, ensure_ascii=False, indent=4))

None


C:\Users\Changda Ma\AppData\Local\Temp\ipykernel_13444\1858372646.py:16: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response_dict = json.loads(pdf_response.json())


In [683]:

import re

def remove_table_and_image_lines(markdown_str: str) -> str:
    """
    Cleans Markdown text:
    1. Removes lines that are likely part of a Markdown table:
       - Separator lines in the format |---|---|---|
       - Lines that start and end with '|' and contain content in between
       - Lines with an excessive number of vertical bars (e.g., more than 3) are considered table lines
    2. Removes Markdown image lines: matches ![...](...)
    3. Removes entire lines starting with Table, Graph, or Fig
    4. Removes parentheses (both English and Chinese) and their contents from each line
    """
    # Match table separator lines like |---|---|---|
    table_sep_regex = re.compile(r'^\s*\|(?:-+\|)+\s*$')
    # Match entire lines that start and end with '|' (possible table rows)
    table_line_regex = re.compile(r'^\s*\|.*\|\s*$')
    # Match Markdown image syntax ![...](...)
    image_line_regex = re.compile(r'!\[.*?\]\(.*?\)')
    # Match lines starting with Table, Graph, or Fig (case insensitive)
    tgf_line_regex = re.compile(r'^\s*(Table|Graph|Fig)\b', re.IGNORECASE)
    # Match parentheses (both English and Chinese) and their contents
    parentheses_regex = re.compile(r'[(（].*?[)）]')
    
    lines = markdown_str.splitlines()
    cleaned_lines = []
    
    for line in lines:
        stripped_line = line.strip()
        
        # 1) Skip lines that are typical table separator lines
        if table_sep_regex.match(stripped_line):
            continue
        # 2) Skip lines that start and end with '|'
        if table_line_regex.match(stripped_line):
            continue
        # 3) Skip lines with too many vertical bars (more than 3)
        if stripped_line.count('|') > 3:
            continue
        # 4) Skip lines containing Markdown image syntax
        if image_line_regex.search(stripped_line):
            continue
        # 5) Skip lines starting with Table/Graph/Fig
        if tgf_line_regex.match(stripped_line):
            continue
        
        # 6) Remove all parentheses and their contents within the line
        cleaned_line = parentheses_regex.sub('', line)
        
        cleaned_lines.append(cleaned_line)
    
    return "\n".join(cleaned_lines)

def remove_tables_images_in_json(input_file: str, output_file: str) -> None:
    """
    Reads JSON data from input_file, iterates through each page's Markdown content,
    cleans the text (removes table rows, image lines, parentheses content, etc.),
    and deletes the 'images' field from pages before writing the cleaned data to output_file.
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if "pages" in data and isinstance(data["pages"], list):
        for page in data["pages"]:
            if "markdown" in page:
                cleaned_markdown = remove_table_and_image_lines(page["markdown"])
                page["markdown"] = cleaned_markdown
            if "images" in page:
                del page["images"]
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def reindex_by_heading(pages: list) -> dict:
    """
    Merges all Markdown text from the pages.
    Whenever a line starts with '#', it is considered a new section.
    Returns a dictionary where the key is the new index, and the value is the corresponding Markdown text.
    """
    # Merge all Markdown content from pages
    combined_markdown = "\n".join(page["markdown"] for page in pages if "markdown" in page)
    sections = {}
    current_section_lines = []
    section_index = 1
    
    for line in combined_markdown.splitlines():
        # If the line starts with '#', consider it a new section
        if line.lstrip().startswith("#"):
            # Save the previous section if it has content
            if current_section_lines:
                sections[str(section_index)] = "\n".join(current_section_lines).strip()
                section_index += 1
                current_section_lines = []
        current_section_lines.append(line)
    
    if current_section_lines:
        sections[str(section_index)] = "\n".join(current_section_lines).strip()
    
    return sections

def process_and_reindex_json(input_file: str, cleaned_file: str, output_file: str) -> None:
    """
    1. First, clean the original JSON file: remove tables, images, parentheses content, etc.,
       and delete the images field, saving the cleaned data to cleaned_file.
    2. Then, read cleaned_file, merge all pages' Markdown content,
       reindex sections whenever encountering a '#' heading,
       and finally generate a new JSON file containing a "sections" field.
    """
    # Clean JSON and generate an intermediate file
    remove_tables_images_in_json(input_file, cleaned_file)
    
    with open(cleaned_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if "pages" in data and isinstance(data["pages"], list):
        sections = reindex_by_heading(data["pages"])
        new_data = {"sections": sections}
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(new_data, f, ensure_ascii=False, indent=4)

# First, clean output.json and save it to cleaned.json,
# then reindex based on '#' headings and output to output_final.json
process_and_reindex_json("output.json", "cleaned.json", "output_final.json")


In [686]:
with open("output_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

if "sections" in data:
   
    data["sections"].pop("1", None)
    data["sections"].pop("5", None)
    data["sections"].pop("8", None)
    data["sections"].pop("14", None)
    data["sections"].pop("21", None)
    data["sections"].pop("25", None)
with open("output_final.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [688]:
with open("output_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

if "sections" in data and "1" in data["sections"]:
    content = data["sections"]["1"]
    marker = "${ }^"
    index = content.find(marker)
 
    if index != -1:
        data["sections"]["1"] = content[:index]

pattern = re.compile(r'\$.*?\$', re.DOTALL)


if "sections" in data:
    for key in data["sections"]:
        section_content = data["sections"][key]
        cleaned_content = pattern.sub("", section_content)
        data["sections"][key] = cleaned_content

with open("output_final.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [690]:
import json
import re

def flexible_regex(phrase: str) -> str:
    escaped = re.escape(phrase)
    pattern = re.sub(r'\\ ', r'\\s+', escaped)
    return pattern

# 要删除的字符串列表
phrases_to_remove = [
    "Introduction",
    "Abstract",
    "METHODS",
    "RESULTS",
    " DISCUSSION",
    "CONCLUSION",
    "Keywords:",
    "Background",
    
]

sentence_to_remove = "Mental health Journal of Exposure Science & Environmental Epidemiology 32:205-212; https://doi.org/10.1038/s41370-021-00349-x"

with open("output_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

if "sections" in data:
    for key in list(data["sections"].keys()):
        content = data["sections"][key]
        for phrase in phrases_to_remove:
            pattern = re.compile(flexible_regex(phrase), re.IGNORECASE)
            content = pattern.sub("", content)
        sentence_pattern = re.compile(flexible_regex(sentence_to_remove), re.IGNORECASE)
        content = sentence_pattern.sub("", content)
        content = re.sub(r'\[.*?\]', '', content)
        content = re.sub(r'\d+', '', content)
        content = re.sub(r'#', '', content)
        if key == "3":
            extra_text = "Mental health Journal of Exposure Science & Environmental Epidemiology 32:205-212; https://doi.org/10.1038/s41370-021-00349-x"
            extra_pattern = re.compile(flexible_regex(extra_text), re.IGNORECASE)
            content = extra_pattern.sub("", content)
            
        data["sections"][key] = content

with open("output_final.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [145]:
from IPython.display import Markdown, display
import json

def display_json_markdown(file_path: str) -> None:
    """
    Reads a JSON file and displays its Markdown content.

    - If the JSON data contains a "pages" list, it iterates through each page.
    - If a page contains a "markdown" field, it displays the Markdown content.
    - If the data format is different, it displays the entire JSON content as Markdown.
    """
    # Read the JSON file
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # If the JSON data contains a "pages" list
    if "pages" in data and isinstance(data["pages"], list):
        for page in data["pages"]:
            if "markdown" in page:
                # Display the Markdown content
                display(Markdown(page["markdown"]))
    else:
        # If the data format is different, display the entire content
        display(Markdown(str(data)))

# Example call: Display the Markdown content from output_clean.json
display_json_markdown("output_clean.json")


# A greener urban environment: Designing green infrastructure interventions to promote citizens' subjective wellbeing 

Pablo Navarrete-Hernandez ${ }^{\mathrm{a}, \mathrm{b}, \mathrm{c}}$, Kate Laffan ${ }^{\mathrm{c}}$<br>* Centre for Sustainable Urban Development , Pontificia Universidad Católica, Chile<br>${ }^{\text {a }}$ Instituto de Estudios Urbanos y Territoriales, Pontifica Universidad Católica, Chile<br>${ }^{\text {b }}$ Department of Psychological and Behavioural Science, London School of Economics and Political Science, United Kingdom

## A R T I C L E I N F O

Keywords:
Green infrastructure
Subjective wellbeing
Urban planning
Happiness
Stress
Safety

## A B STRACT

Research using subjective wellbeing  measures finds that the greener an individual's local environment is, the higher the levels of happiness and the lower levels of stress they tend to report experiencing. This literature presents positive associations between existing large-scale green spaces, such as urban parks or squares, and the wellbeing of residents living in close proximity to them. In contrast, in the current work, we present a novel approach which combines SWB measures with photo simulation in order to examine the impact of street-level green infrastructure interventions on the people's perceptions of the SWB associated with urban sites. We tested the approach with the attendees of the 20th Biennale on Architecture and Urbanism in Chile in 2017, exploring the impact of four different types of street-level green interventions. The results indicate that all types of green interventions considered significantly increase the perceived happiness and reduce the perceived stress associated with the sites during short exposures, with varying effect sizes across different types and scales of interventions. The proposed technique could be used in urban planning processes to examine the potential SWB benefits of green infrastructure investments prior to their being rolled out.

## 1. Introduction

Over half of the world's population now live in urban areas and this proportion is predicted to grow rapidly over the next 30 years . While there are benefits to living in towns and cities compared to rural areas, including enhanced economic and social opportunities, a major downside of life in many urban areas around the world is the relatively low level of green space that residents have access to . In addition to the urban-rural divide, inequalities in access to green space exist within cities too: recent research indicates that those with low socioeconomic status have less access to high-quality green space in the Global North and to green space of any kind in the Global South than their economically better off counterparts . These inequalities go beyond green spaces and also apply to small-scale green elements such as the trees and shrubs . As towns and cities around the world grow to accommodate more and more people, a key priority for urban planners is the greening of urban spaces in ways that effectively promote the wellbeing of all residents.

Capturing the value of greenery for the wellbeing of urban residents
is central to efforts to encourage its provision. However, this is no easy task. First, like many other elements of environmental quality, such as air and noise pollution, elements of green infrastructure, including street trees, playing fields, parks and other open green spaces, are not typically bought and sold in markets . Second, wellbeing itself is a broad and multidimensional construct which covers how well individuals are doing in their lives across material, social, health and subjective dimensions . How best to operationalise wellbeing in order to examine how it is impacted by green space is, therefore, open to debate . It has been suggested that the resulting difficulties in assessing the overall value of urban green space are to blame for the widespread underinvestment in green infrastructure and the encroachment of urban sprawl into green spaces in cities around the world .

Traditional approaches to valuing the benefits of green space have centred on using people's preferences as measures of their wellbeing, in accordance with a preference satisfaction account of wellbeing that says that people are well when they get what they prefer . Such approaches include both revealed and stated preference

[^0]
[^0]:    ${ }^{\text {a }}$ Corresponding author. Tel.: +56223545640.

    E-mail address: pablo.navarrete@uc.cl .

methods . Revealed preference studies look to how people behave in markets for related goods to infer their preferences for, and therefore the value of, green spaces. Hedonic price estimates, for example, are based on the premium associated with external non-market factors such as green space in related markets such as housing or labour markets . Stated preferences, on the other hand, bypass real markets and value green spaces based on what individuals say they would be willing to pay to access it, or willing to accept to forgo doing so, most often based on hypothetical scenarios . These standard methods have been critiqued, however, for failing to fully capture the wide range of positive health, ecosystem services and wellbeing benefits that green space gives rise to .

In recent years, and in response to the challenges levelled at standard preference-based methods of valuation, researchers have begun to examine the benefits of green infrastructure using non-preference based measures of wellbeing. While some of this work has focused on documenting the links between green space and objective health and wellbeing indicators such as body mass index and cortisol levels , other work has looked to measures of subjective wellbeing  - individuals' own reports of how they think about their lives and feel as they go about them  - to investigate the value of green space. This work finds, for example, that living in a greener environment is linked to higher levels of life satisfaction and happiness and lower levels of stress .

Of particular importance, is evidence from the literature using both objective wellbeing indicators and SWB measures which indicates that more green space is linked to lower levels of stress in deprived urban communities and that green space can act as a buffer for stressful life events . These findings suggest that access to green space may be of particular importance for the wellbeing of the most vulnerable in society. This evidence has strengthened the case for integrative sustainability policies which look to promote social as well as ecological sustainability in urban communities by greening urban spaces in ways that limit the potential risks of gentrification that green infrastructure investment can pose .

While demonstrating the importance of green space for wellbeing, research using objective and SWB measures has had far less to say about how best to channel green investments so as to maximise their positive impact on urban residents. Additionally, with the exception of a few studies which have considered the impact of street-level green infrastructure, such as street trees, on residents' levels of neighbourhood satisfaction , the SWB work that does exist has tended to focus on large-scale green spaces with substantial land requirements . Understanding how to design small-scale green infrastructure in ways that can promote wellbeing is of particular importance in urban areas around the world where such infrastructure must be incorporated in built-up city areas where public land is costly and scarce.

In the current work, and with a view to contributing to these gaps and expanding current techniques in green infrastructure planning and design, we use photo simulation in combination with SWB measures to study the impact of street-level green infrastructure interventions  on people's perceptions of the SWB
associated with urban sites prior to their being rolled out. More specifically, we explore the impact of 'greening' tram lines, pedestrian streets, avenues and small streets on the feelings of happiness and stress that people associate with images of urban sites. Additionally, we compare our SWB results to those relating to a third outcome variable which has been considered in some of the existing photo simulation literature - individuals' perceptions of the safety of sites .

We carried out the study with attendees to the 20th Biennale on Architecture and Urbanism in Chile in 2017. The results of the study indicate that all of the different types of green interventions we consider increase the average ratings of happiness and reduce the average ratings of stress associated with the images of the sites, with the magnitude of the impact varying across the different types and scales of green interventions. Moreover, we find that people's perceptions of safety are less consistently impacted by green interventions than are their ratings of SWB. Overall, the study demonstrates the benefits of using photo simulation in conjunction with SWB measures, in addition to more traditional measures like people's perceptions of safety, to compare and contrast the potential benefits of different types of green interventions.

In what follows, we first review the existing literature on SWB and green space and the use of photo simulation in the urban planning literature. We then go on to present our new approach which builds on these two, thus far, separate bodies of literature.

### 1.1. SWB and green space

SWB is an umbrella term for how people think and feel as they go about their lives . It includes both evaluative measures of SWB, such as people's overall evaluations of their lives and of domains of their lives such as their neighbourhood satisfaction, as well as experiential measures of SWB which reflect their affective experiences, such as the levels of happiness and stress they experience day to day . SWB measures have gained prominence in both economics and psychology over the past two decades and are now widely understood to provide valuable and reliable information about how individuals' lives are going . These measures are also increasingly being used to inform public policy. For example, many national statistical agencies, including those in France, Italy, Canada, New Zealand, the UK and Australia, are now collecting and using SWB data to monitor national wellbeing. In 2011, the UK government's Green Book - its guidance on cost-benefit analysis - was amended to include recommendations on the use of SWB measures in the valuation of non-market goods like green space .

In recent years, a burgeoning body of literature has emerged which uses SWB measures to document the benefits of green infrastructure for individual wellbeing . Existing correlational evidence from this literature indicates that living in environments with higher levels of green space is associated with lower levels of depression, stress and anxiety , and higher levels of happiness and life satisfaction . For example, in a nationally representative longitudinal sample of UK residents, White et al.  find that, controlling for individual and regional covariates, individuals report both lower mental distress and higher life satisfaction on average when they live in urban areas with relatively high levels of green space. Krekel, Kolbe, and Wüstemann  find similar results when looking at the relationship between green space in urban areas in Germany, while also documenting a particularly strong positive relationship for older residents.

There is also some experimental work which suggests that green infrastructure may have a causal impact on how people feel. For example, some small-scale work finds that incorporating natural elements

## TYPE OF STREET INTERVENTION

1. Green Tram Lines 
2. Green Tram Lines 
3. Green Tram Lines 
4. Green Pedestrian Streets 
5. Green Pedestrian Streets 
6. Green Pedestrian Streets 

BEFORE IMAGE

AFTER IMAGE

into individuals' indoor environments, for example, by adding plants to their office places, classrooms or hospital waiting rooms, can serve to enhance their SWB . Other studies compare the impact of walking in natural versus built-up environments, finding that walking in green space results in higher levels of happiness and lower levels of anger than doing so in built-up environments . All of this evidence suggests that policymakers should take seriously the potential wellbeing benefits of increasing the amount of green space citizens are exposed to in their daily lives.

While the above work provides insights into the value of having green infrastructure nearby, it cannot shed light on how to invest in such infrastructure so as to maximise its potential wellbeing benefits. In order to address this, the current work goes a step further in the use of SWB measures by integrating them into the planning process with the help of photo simulation.

### 1.2. Photo simulation

Photo simulation  is a technique that has been widely used in planning research and environmental psychology . It involves adapting photographs of sites so that they incorporate different changes to the physical environment under consideration. PS has been used to capture the impact of different physical interventions on people's preferences. For example, one study explored residents' preferences over a commercial strip development and used photo simulation to explore the impact of including a car park and natural elements, such as trees and flowers, on people's preferences over the development . Another study based in Switzerland explored how different forms of river restoration impacted upon the extent to which individuals perceived it satisfied their needs, among other outcomes . Photo simulation has also been used in a recent study to explore the impact of different architectural design features of care homes on older adults' attitudes toward these facilities .

Another line of research, relating specifically to green infrastructure, has used photo simulation to explore the impact of the addition of green elements to physical environments on people's perceptions of sites, with a particular focus on perceptions of safety. In an early study, Kuo and colleagues  used PS to examine the impact of tree placement, tree density and levels of grass maintenance on Chicago residents' preferences and sense of safety of their local area, finding that both tree density and grass maintenance positively impacted the outcomes in question. A more recent study by Jiang and colleagues  used PS to investigate the impact of green interventions on people's perceptions of the safety of alleyways. They found that green interventions involving geometric vegetation increased perceptions of safety to a greater extent than did naturalistic vegetation. Finally, Nejati, Shepley, Rodiek, Lee, and Varni  used PS to demonstrate how adding natural elements to hospital staff break areas positively impacted upon staff members' perceptions of the restorative nature of these areas. This literature demonstrates that this technique can inform our understanding of individuals' perceptions of, as well as preferences over, sites. Moreover, PS can produce insights which inform the design and maintenance of both indoor and outdoor green infrastructure. In the current work, we use PS in conjunction with SWB measures to examine the potential benefits of urban green interventions.

## 2. Methods

We demonstrate our proposed approach in a study constructed using a specially developed online platform - www.urban-experiment.com - and implemented in person with attendees to the 20th Biennale of Architecture and Urbanism in Valparaiso in 2017.

### 2.1. Study design

We designed the study to gather data on individuals' perceptions of a series of computerized visual simulations of street-level green interventions . We chose to use photo simulation, instead of other more technologically advanced methods such as 360-degree videos, as it represents a very low-cost technique to generate insights into the green interventions under consideration. In the planning process, it is typical for urban planners, architects and others to produce computerized images of their proposed interventions. In this study, we demonstrate how such images could be used in combination with images of the existing sites to gain low-cost insights into how the interventions might impact people's perceptions of those sites. While generating 360degree videos of existing sites is affordable, generating a 360-degree simulated video of high enough quality to render the green interventions believable is still relatively expensive and time-consuming. With a view to maximising the potential uptake of our approach among practitioners working on green infrastructure projects in both cities in the developing and developed world, we therefore opted to use photo simulation.

We briefly expose participants to images urban sites either before or after the incorporation of a green intervention. They rated these images according to the happiness and stress they associated with them on a ten-point scale: from completely unhappy to completely happy and completely stressed to completely relaxed, respectively. Self-reports of happiness and stress represent affective measures of SWB that assess individuals' experiential rather than evaluative wellbeing. These two types of affect have dominated the SWB-green space literature to date . In the current work we adapt SWB measures that have been widely used to investigate people's affective wellbeing on the previous day  or in the moment using experience sampling . In addition to the two SWB measures, we also asked individuals to report their perceptions of the safety of the
sites in question. As previously discussed, individuals' perceptions of safety commonly feature in the photo simulation literature.

We used twelve pairs of images of urban sites before and after the incorporation of vegetation in the study. All images represent urban areas in France. The images represent four different types of green interventions: 1) green tram lines ; 2) green pedestrians streets ; 3) green avenues ; and 4) small green streets .

We further categorised image types into three different scales of green interventions: small, medium and large. We modified the before and after images slightly to hold constant important features of the images  between the before and after pairs of images. As a result, we obtained twelve pairs of computerized images with each pair being equal in all its components but the incorporation of street vegetation.

### 2.2. Sampling method

We installed a stand at the Biennale of Architecture and Urbanism from Friday to Sunday over the last week of October and the first week of November 2017. We chose this site as it gave us access to a convenience sample with which to demonstrate our approach of combining photo simulation with SWB measures to inform urban planning and design decisions. We specifically targeted this event as we expected to find enthusiastic volunteers among the attendees who would participate in our study without material incentive to do so. We were proven correct: of a total of 252 walking-by visitors who stopped at the urban experiment stand and we asked to participate in the study, 240 accepted. We approached potential participants and gave them a flyer about the urban experiment platform and asked them to participate in a 6-minute long study on urban perceptions. No monetary incentives or gifts were offered for participation. Once they had agreed to participate, the attendees completed an online registration process which included a questionnaire about their socio-demographic information , a brief about the study protocol and an informed consent form.
"After registering, we presented participants with a sequence of 12 images of urban sites on a 24 -inch screen in full-screen display mode. Participants saw each street image for a minimum of 6 s before rating the happiness and stress they associated with the site, as well as their perceptions of its safety . Participants took an average of $4: 12 \mathrm{~min}$ to rate the series of 12 images we presented to them."

To ensure that the covariates were balanced between participants in the before and after groups, we applied a double randomization process using our computer programme: a random image-order assignment and a random before-or-after assignment. First, the programme randomly allocated the order of the twelve pairs of images for each participant. This allows us to control for potential spillover effects or fatigue affecting participants' responses from one picture to the next. Second, for each pair of images, the programme randomly assigned the participant to see either a before or after image. This double random assignment generated a unique order of appearance and combination of before/ after images for each respondent. Prior to conducting our analysis, we carried out balance tests at the image level to investigate whether there were differences across the before/after groups in the observed characteristics age, gender, student status, household size, nationality, subject of study, native language, educational level, employment status, image order, day of the test and hour of the test. Across the 156 tests we carried out only 5 were significant at the $5 \%$ level indicating the the randomisation was successful .

TYPE OF STREET INTERVENTION
7. Green Avenues 
8. Green Avenues 
9. Green Avenues 
10. Small Green Streets 
11. Small Green Streets 
12. Small Green Streets 

BEFORE IMAGE


Descriptive Statistics.


### 2.3. Data

To perform the analysis, we use three data sources collected during the study: 1) background characteristics, 2) measures relating to the experimental conditions and 3) participants' subjective perceptions. Participants' background characteristics include their age, gender, student status, household size, nationality, the subject of study, native language, educational level and employment status. The experimental condition measures include treatment status, the date and the time of participants' responses, image order and the size of the treated area. Subjective perception measures involved the declared perceptions of happiness, stress and safety for each image. Table A1 in the Appendices provides a detailed description of the measures used to assess these characteristics.

...

the literature as to whether SWB measures should be aggregated and if so what weighting should be given to each of the separate measures . Although the results suggest that the green interventions explored have significant positive impacts on happiness and negatives impacts on stress, it is possible that other urban interventions might increase perceptions of both happiness and stress. It is also possible that green interventions would have a negative impact on other aspects of SWB while benefiting people's happiness and stress perceptions. In order to explore these issues further, future work should look to include a wide range of theoretically-relevant SWB variables to explore the extent to which the impact of green interventions varies depending on the SWB measures under consideration.

Relatedly, the approach does not produce monetary values associated with the various interventions which can feed into cost-benefit analysis. In contrast, stated preference methods can be used to produce monetary estimates of the value that local residents place on a green infrastructure project based on how much they say they would be willing to pay for it. In future, photo simulation could be incorporated into willingness to pay studies to estimate the benefits of green interventions using the before and after technique used in the current work. If such an approach was developed, it would be interesting to examine the associations between willingness to pay reports and SWB perceptions to see which perceptions are most predictive of the monetary values people place on the green interventions under consideration.

Another limitation of this study relates to the fact that participants were only exposed to visual stimuli. Existing research indicates that combining street images with soundscapes can enhance the impact of nature simulations on people's wellbeing . In line with this, future studies should include street background noise to further our understanding of how green interventions may shape people's perceptions of the wellbeing they associate with urban sites.

A final limitation of the study is that although this approach measures the impact of green interventions on people's perceptions, holding constant the contents of the images of the sites  outside of the intervention area, this technique cannot account for what is replaced by vegetation in the after images in the section of the image that includes the intervention. While is important to recognise this as a limitation, it is also worth highlighting that the approach has external validity in that real green interventions necessarily take up space that could otherwise be used differently, for example for parking or road space.

The approach detailed in this work has the potential to feed into green infrastructure policy-making processes. It could be adopted by urban planners when looking to make green infrastructure investments to compare the impact of a range of different green investments across different sites on perceptions of the SWB associated with the site. Importantly, this approach should be incorporated into integrative sustainability policies which look to promote social as well as ecological sustainability in urban communities, taking into account the risks of gentrification that greening urban spaces can pose . One way in which this approach could contribute to these efforts would be to carry out this exercise with local residents in order to ensure that the perceptions of those who will be exposed to the intervention are taken into account. This is of particular importance given that existing research indicates that perceptions of the social environment interact with perceptions of the physical environment . For example, Chong et al.  found that the positive relationship between green space and

SWB wellbeing is reversed in deprived neighbourhoods where residents perceive the outdoor environment to be unsafe. The current technique could ensure that local voices are included in the planning process and that the important interactions between individual characteristics, the social environment and the physical environment are accounted for.

This technique could also be used by urban planners in ways which the current study does not directly speak to. For example, this approach could be used to test different green infrastructure project options for a single site. In this case, residents could be randomly presented with a range of different intervention options to establish which of them has the greatest positive impact on the residents' perceptions. This approach could also be used to explore the impact of other characteristics of green interventions which preference-based studies have identified as being important, such as whether the green space is protected or provides recreational amenities  and to investigate the impact of physical changes to urban sites other than those involving green elements. The online platform built for this study - www.urbanexperiment.com - could facilitate urban planners in carrying out these various form of analysis.

## 5. Conclusion

In this study, we present a new approach to examining the impact of green interventions on people's perceptions of urban sites. The technique involves randomly presenting participants with photos of urban sites before or after the incorporation of a green intervention and eliciting participants' perceptions of the SWB they associate with the site. The study involved collecting over 5000 ratings of images from 240 attendees of the 20th Architecture and Urbanism Biennale in Chile in 2017, which were used to explore the impact of street-level green interventions on people's perceptions of happiness and stress. With a view to linking the findings to existing photo simulation work, people's perceptions of the safety of the sites under consideration were also collected. The results show that street-level green interventions have a positive impact on the feelings of happiness and stress people associate with urban sites and that the impacts vary by the type of green intervention. The results relating to these two SWB measures are substantively different from those produced from using measures of people's perceptions of safety.

SWB measures can be used in combination with photo simulation by urban planners looking to gain insights into the SWB impacts of various green infrastructure investment options across multiple sites. It can also be used to compares the different interventions options relating to a single site and the impact of other forms of infrastructure changes such as new commercial developments and public spaces. By placing people's SWB at the centre of infrastructure developments, the approach can feed into decisions about the greening of urban areas in order to maximise the benefits to people living in urban environments.

## Acknowledgments

We would like to thank Dr. Nicolas Navarrete, Dr Milt Mavrakakis, Joel Suss and Dr. Davide Lucca for their helpful methodological advice and participants at CEDEUS paper session for their comments and suggestions. We also thank Felo Paredes for his help with the data collection at the 20th Biennale on Architecture and Urbanism in Chile. We are grateful to the anonymous referees for their constructive input which greatly help to enhance the quality of the paper.

Measures.


The impact of all types of green interventions on perceptions of happiness, stress and safety.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

The impact of green tram lines on perceptions of happiness, stress and safety Tram.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

The impact of green pedestrian streets on perceptions of happiness, stress and safety.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

The impact of green avenues on perceptions of happiness, stress and safety.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

The impact of small green streets on perceptions of happiness, stress and safety.


Robust standard errors in parentheses.
**p $<.05$.
***p $<0.1$.

Exploring how the scale of the green interventions relates to their impact on perceived levels of happiness.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

Exploring how the scale of the green interventions relates to their impact on perceived levels of stress.


Robust standard errors in parentheses.
**p $<.05$.
***p $<.01$.

Exploring how the scale of the green interventions relates to their impact on perceptions of safety.


Robust standard errors in parentheses.
${ }^{* *} \mathrm{p}<.05$.
${ }^{* * *} \mathrm{p}<.01$.

## Appendix A. Supplementary data

Supplementary data to this article can be found online at https://doi.org/10.1016/j.landurbplan.2019.103618.

## References

Annerstedt, M., Jönsson, P., Wallergård, M., Johansson, G., Karlson, B., Grahn, P., ... Währborg, P. . Inducing physiological stress recovery with sounds of nature in a virtual reality forest - Results from a pilot study. Physiology Behavior, 118, 240-250.
Ambrey, C. L. . Urban greenspace, physical activity and wellbeing: The moderating role of perceptions of neighbourhood affability and incivility. Land Use Policy, 57, 638-644.
Ambrey, C. L., Fleming, C. M., \& Manning, M. . Greenspace and life satisfaction: The moderating role of fear of crime in the neighbourhood. Nature Conference. Opportunities for the Critical Decade: Enhancing wellbeing within Planetary Boundaries . Canberra, Australia: University of Canberra.
Atkinson, G., \& Mouraio, S. . Environmental cost-benefit analysis. Annual Review of Environment and Resources, 33, 317-344.
Atkinson, G., Bateman, I., \& Mouraio, S. . Recent advances in the valuation of ecosystem services and biodiversity. Oxford Review of Economic Policy, 28, 22-47.
Bettencourt, L., \& West, G. . A unified theory of urban living. Nature, 467, 912.

Beukeboom, C. J., Langeveld, D., \& Tanja-Dijkstra, K. . Stress-reducing effects of real and artificial nature in a hospital waiting room. The Journal of Alternative and Complementary Medicine, 18, 329-333.
Brander, L. M., \& Koetse, M. J. . The value of urban open space: Meta-analyses of contingent valuation and hedonic pricing results. Journal of Environmental Management, 92, 2763-2773.
Cerina, V., Fornara, F., \& Manca, S. . Architectural style and green spaces predict older adults' evaluations of residential facilities. European Journal of Ageing, 14, 207-217.
Chong, S., Lobb, E., Khan, R., Abu-Rayya, H., Byun, R., \& Jalaludin, B. . Neighbourhood safety and area deprivation modify the associations between parkland and psychological distress in Sydney, Australia. BMC Public Health, 13, 422.
Choumert, J., \& Salanié, J. . Provision of urban green spaces: Some insights from economics. Landscape Research, 33, 331-345.
De Vries, S., Verhetj, R. A., Groenewegen, P. P., \& Spreeuwenberg, P. . Natural environments-healthy environments? An exploratory analysis of the relationship
between greenspace and health. Environment and Planning A, 35, 1717-1731.
Dolan, P., \& Metcalfe, R. . Measuring subjective wellbeing: Recommendations on measures for use by national governments. Journal of Social Policy, 41, 409-427.
Dolan, P., Peungood, T., \& White, M. . Do we really know what makes us happy? A review of the economic literature on the factors associated with subjective wellbeing. Journal of Economic Psychology, 29, 94-122.
Gerrish, E., \& Watkins, S. L. . The relationship between urban forests and income: A meta-analysis. Landscape and Urban Planning, 170, 293-308.
Fleming, C. M., Manning, M., \& Ambrey, C. L. . Crime, greenspace and life satisfaction: An evaluation of the New Zealand experience. Landscape and Urban Planning, 149, 1-10.
Fong, K. C., Hart, J. E., \& James, P. . A review of epidemiologic studies on greenness and health: Updated literature through 2017. Current Environmental Health Resports, 3, 77-87.
Fujiswara, D., \& Campbell, D. . Valuation Techniques for Cost Benefit Analysis: Stated Preference, Revealed Preference and Subjective Well-Being Approaches, HM Treasury Report. Available for download at: https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/209107/greenbook_val uationtechniques.pdf.
Hartig, T., Evans, G. W., Jamner, L. D., Davis, D. S., \& Gärling, T. . Tracking restoration in natural and urban field settings. Journal of Environmental Psychology, 23, 109-123.
Hazer, M., Formica, M. K., Dieterlen, S., \& Morley, C. P. . The relationship between self-reported exposure to greenspace and human stress in Baltimore, MD. Landscape and Urban Planning, 169, 47-56.
Hur, M., \& Morrow-Jones, H. . Factors that influence residents' satisfaction with neighborhoods. Environment and Behavior, 40, 619-635.
Irwin, E. G. . The effects of open space on residential property values. Land Economics, 78, 465-480.
Jiang, B., Mak, C. N. S., Larsen, L., \& Zhong, H. . Minimizing the gender difference in perceived safety: Comparing the effects of urban back alley interventions. Journal of Environmental Psychology, 51, 117-131.
Jim, C. Y., \& Chen, W. Y. . Impacts of urban environmental elements on residential housing prices in Guangzhou . Landscape and Urban Planning, 78, 422-434.
Jorgensen, A., Hitchmough, J., \& Calvert, T. . Woodland spaces and edges: Their impact on perception of safety and preference. Landscape and Urban Planning, 60,

135-150.
Junker, B., \& Buchecker, M. . Aesthetic preferences versus ecological objectives in river restorations. Landscape and Urban Planning, 85, 141-154.
Kearney, A. R. . Residential development patterns and neighborhood satisfaction: Impacts of density and nearby nature. Environment and Behavior, 38, 112-139.
Kondo, M., Fluehr, J., McKeon, T., \& Branas, C. . Urban green space and its impact on human health. International Journal of Environmental Research and Public Health, $15, 445$.
Kong, Faidhua, Yin, Haiwei, \& Nakagoshi, Nobukazu . Using GIS and landscape metrics in the hedonic price modeling of the amenity value of urban green space: A case study in Jinan City, China. Landscape and Urban Planning, 79, 240-252.
Krefts, A., Augustin, M., Schlinzen, K., Oßenbrügge, J., \& Augustin, J. . How does the urban environment affect health and well-being? A systematic review. Urban Science, 2, 21.
Krekel, C., Kolbe, J., \& Wüstemann, H. . The greener, the happier? The effect of urban land use on residential well-being. Ecological Economics, 121, 117-127.
Küller, R., \& Lindsten, C. . Health and behavior of children in classrooms with and without windows. Journal of Environmental Psychology, 12, 305-317.
Kuo, F. E., Bacaicoa, M., \& Sullivan, W. C. . Transforming inner-city landscapes: Trees, sense of safety, and preference. Environment and Behavior, 30, 28-59.
Kweon, B. S., Ellis, C. D., Leiva, P. I., \& Rogers, G. O. . Landscape components, land use, and neighborhood satisfaction. Environment and Planning B: Planning and Design, $37, 500-517$.
Lachowycz, K., \& Jones, A. P. . Greenspace and obesity: A systematic review of the evidence. Obesity Reviews, 12, e183-e189.
Larsen, L., Adams, J., Deal, B., Kweon, B. S., \& Tyler, E. . Plants in the workplace: The effects of plant density on productivity, attitudes, and perceptions. Environment and Behavior, 30, 261-281.
Maas, J., Spreenseenberg, P., Van Winsum-Westra, M., Verheij, R. A., Vries, S., \& Groenewegen, P. P. . Is green space in the living environment associated with people's feelings of social safety? Environment and Planning A, 41, 1763-1777.
MacKerran, G., \& Mourato, S. . Happiness is greater in natural environments. Global Environmental Change, 23, 992-1000.
Mazzotta, M. J., Besedin, E., \& Speers, A. E. . A meta-analysis of hedonic studies to assess the property value effects of low impact development. Resources, 3, 31-61.
Mell, I. C., Henneberry, J., Hehl-Lange, S., \& Keskin, R. . Promoting urban greening: Valuing the development of green infrastructure investments in the urban core of Manchester, UK. Urban Forestry \& Urban Greening, 12, 296-306.
Mullenbach, L. E., \& Baker, B. L. . Environmental justice, gentrification, and leisure: A systematic review and opportunities for the future. Leisure Sciences, 1-18.
Nejati, A., Shepley, M., Rodiek, S., Lee, C., \& Varni, J. . Restorative design features for hospital staff break areas: A multi-method study. HEBD: Health Environments Research \& Design Journal, 9, 16-35.
Nisbet, E. K., \& Zelenski, J. M. . Underestimating nearby nature: Affective forecasting errors obscure the happy path to sustainability. Psychological Science, 22, $1101-1106$.
Nowak, D. J., Hirshayashi, S., Bodine, A., \& Greenfield, E. J. . Tree and forest effects on air quality and human health in the United States. Environmental Pollution, 193, 119-129. https://doi.org/10.1016/j.eevpol.2014.05.028.
Panduro, T. E., \& Vcie, K. L. . Classification and valuation of urban green spaces-A hedonic house price valuation. Landscape and Urban Planning, 120, 119-128.

Pedlowski, M. A., Da Silva, V. A. C., Adell, J. J. C., \& Heynen, N. C. . Urban forest and environmental inequality in Campos dos Goytacazes, Rio de Janeiro, Brazil. Urban Ecosystems, 6, 9-20.
Pondyal, N. C., Hodges, D. G., \& Merrett, C. D. . A hedonic analysis of the demand for and benefits of urban recreation parks. Land Use Policy, 26, 975-983.
Rigolon, A. . A complex landscape of inequity in access to urban parks: A literature review. Landscape and Urban Planning, 153, 160-169.
Rigolon, A., Browning, M., Lee, K., \& Shin, S. . Access to urban green space in cities of the global south: A systematic literature review. Urban Science, 2, 67.
Rodiek, S. D., \& Fried, J. T. . Access to the outdoors: Using photographic comparison to assess preferences of assisted living residents. Landscape and Urban Planning, 73, 184-199.
Scarpia, R., Hutchinson, W. G., Chilton, S. M., \& Buongiorno, J. . Importance of forest attributes in the willingness to pay for recreation: A contingent valuation study of Irish forests. Forest Policy and Economics, 1, 315-329.
Stone, A. A., \& Mackie, C. E. . Subjective well-being: Measuring happiness, suffering, and other dimensions of experience. National Academies Press.
Sullivan, W. C., \& Lovell, S. T. . Improving the visual quality of commercial development at the rural-urban fringe. Landscape and Urban Planning, 77, $152-166$.
Thompson, C. W., Roe, J., Aspinall, P., Mitchell, R., Clow, A., \& Miller, D. . More green space is linked to less stress in deprived communities: Evidence from salivary cortisol patterns. Landscape and Urban Planning, 105, 221-229.
Thompson, C. W., Aspinall, P., Roe, J., Robertson, L., \& Miller, D. . Mitigating stress and supporting health in deprived urban communities: The importance of green space and the social environment. International Journal of Environmental Research and Public Health, 13, 440.
Tyrväinen, L. . Economic valuation of urban forest benefits in Finland. Journal of Environmental Management, 62, 75-92.
UN DESA  Revision 2018 World Urbanization Prospects. United Nations Department of Economics and Social Affairs. Retrieved from https://www.un.org/ development/desa/publications/2018-revision-of-world-urbanization-prospects. html.
Van den Berg, A. E., Maas, J., Verheij, R. A., \& Groenewegen, P. P. . Green space as a buffer between stressful life events and health. Social Science \& Medicine, 70, $1203-1210$.
White, M. P., Alcock, I., Wheeler, B. W., \& Deplodge, M. H. . Would you be happier living in a greener urban area? A fixed-effects analysis of panel data. Psychological Science, 24, 920-928.
WHO . Urban green spaces: A brief for action. Available for download: World Health Organisation Report at: Http://www.euro.who.int/_data/assets/pdf_file/0010/ 342289/Urban-Green-Spaces_EN_WHO_web.pdf?ua $=1$.
Wolch, J. R., Byrne, J., \& Newell, J. P. . Urban green space, public health, and environmental justice: The challenge of making cities 'just green enough'. Landscape and Urban Planning, 125, 234-244.
Wolf, K. L., \& Robbins, A. . Metro nature, environmental health, and economic value. Environmental Health and Preventive Medicine. https://doi.org/10.1289/ehp. 1408216.

Zhang, Y., Van den Berg, A., Van Dijk, T., \& Weitkamp, G. . Quality over quantity: Contribution of urban green space to neighborhood satisfaction. International Journal of Environmental Research and Public Health, 14, 535.